In [2]:
import numpy as np
import cvxpy as cp
import wta
np.set_printoptions(precision=3, suppress=True)

In [3]:
wpns = 6
tgts = 3
# Define the y matrix
y = np.zeros((tgts, wpns))
y[0,:] = [0.5, 0.2, 0.3, 0.3, 0.1, 0.6]
y[1,:] = [0.1, 0.2, 0.3, 0.3, 0.6, 0.2]
y[2,:] = [0.2, 0.21, 0.31, 0.31, 0.6, 0.65]

V0 = 6
q0 = np.array([0.8, 0.8, 0.4, 0.45, 0.5, 0.05])


In [6]:
def refProx(i, q, y, V, verbose=False):
    tgts, wpns = y.shape
    
    x = cp.Variable(y.shape, nonneg=True)
    econs = [cp.sum(x, axis=0) == 1]
    z1 = cp.exp(x[i,:]@cp.log(q))
    obj1 = cp.Minimize(V*z1 + 0.5*cp.sum_squares(x-y))
    prob1 = cp.Problem(obj1, econs)
    prob1.solve(verbose=verbose)
    print(obj1.value)
    return x.value

In [17]:
# Get the reference proximal operator solution
x_ref = refProx(0, q0, y, V0)
print(x_ref)

0.41069458918430923
[[0.597 0.36  0.454 0.438 0.094 0.78 ]
 [0.152 0.315 0.268 0.276 0.453 0.   ]
 [0.252 0.325 0.278 0.286 0.453 0.22 ]]


In [13]:
def fastProx(i, q, y, V, verbose=False):
    tgts, wpns = y.shape
    x = np.zeros((tgts, wpns))
    y_sum = np.sum(y, axis=0)-y[i,:]
    # Find values of x[i,:] to set to 1 because the benefit, 
    # even when all other values of x[i,:] are 1,
    # exceeds the cost of setting it to 1

    # Find the values of x[i,:] to set to 0 because the benefit of setting it to a non zero value
    # even when all other values of x[i,:] are 0,
    # is exceeded by the cost of setting it to a non zero value

    # Solve the problem for the remaining values of x[i,:]
    promising = [i for i in range(wpns)]
    base_val = V
    promising_vals = singleWTA(q, base_val, y[i,:], y_sum, tgts-1, promising, True)
    offset = ((1-promising_vals)-y_sum)/(tgts-1)
    x = y + offset
    x[i, promising] = promising_vals
    return x



In [14]:
def singleWTA(q, V, y_i, y_sum, n, promising, verbose=False):
    '''
    Solve the weapon-target assignment problem for a single target
    Inputs:
    q: (m,) array of survival probabilities
    V: target value (float)
    cost_diff: (m,) array of the difference between the cost of each weapon and the base value
    promising: list of promising weapons
    Outputs:
    p: list of weapons to use
    '''
    m = len(promising)
    small_q = q[promising]
    small_y = y_i[promising]
    small_c = y_sum[promising]
    print("small_q", small_q)
    print("small_c", small_c)
    print("V", V)
    x = cp.Variable(m, nonneg=True)
    ww = x@cp.log(small_q)
    ss = 0.5*(cp.sum_squares(x - small_y) + cp.sum_squares(((1-x) - small_c)/n))
    cons = [x <= 1]
    obj = cp.Minimize(V*cp.exp(ww) + ss)
    prob = cp.Problem(obj)
    prob.solve()
    if verbose:print("prob status", prob.status)
    if verbose:print("prob value", prob.value)
    if verbose:print("prob x", x.value)
    used = [i for i in range(m) if x.value[i] > 0.5]
    print(used)
    return x.value



In [15]:
# Get the fast proximal operator solution
x = fastProx(0, q0, y, V0)
print(x)

small_q [0.8  0.8  0.4  0.45 0.5  0.05]
small_c [0.3  0.41 0.61 0.61 1.2  0.85]
V 6
prob status optimal
prob value 0.3118355092153159
prob x [0.568 0.306 0.432 0.417 0.126 0.882]
[0, 5]
[[ 0.568  0.306  0.432  0.417  0.126  0.882]
 [ 0.166  0.342  0.279  0.286  0.437 -0.166]
 [ 0.266  0.352  0.289  0.296  0.437  0.284]]


In [18]:
x_ref


array([[0.597, 0.36 , 0.454, 0.438, 0.094, 0.78 ],
       [0.152, 0.315, 0.268, 0.276, 0.453, 0.   ],
       [0.252, 0.325, 0.278, 0.286, 0.453, 0.22 ]])